## DataSet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
from glob import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import itertools
import numpy as np
from PIL import Image
T = torch.Tensor
rand_m = np.random.random

In [2]:
from data import Tiny_imagenet_dataset

In [3]:
tt = Tiny_imagenet_dataset()

In [4]:
train_loader = DataLoader(tt, batch_size=100, shuffle=True)

In [5]:
for x, y in train_loader:
    break

In [6]:
y

tensor([ 49,  29,  87,  22, 112, 163,  28, 191,  84,  66,  11,  18,  75, 127,
         56, 156,  69, 110, 147, 184,  73, 103, 159, 163,   1, 167,  90,  14,
        125,  51,  69,  26, 100, 187, 136,   5,  44,  77,  97,  80, 190,  89,
         45, 128,  33,  84,  56,  44,  18, 187,  59,  16,  53, 194, 172, 107,
         65,  93,   9,  97, 188, 128,  26,  11, 180, 109,  41, 153,  85, 176,
        181, 137,  78,  27,   5, 128,  39, 162, 164, 160,  77,  26, 172,  92,
        115,  41, 103,  82, 153,  97,  42, 114, 121, 114,  65,  27, 191, 189,
          0, 133])

In [7]:
from torchvision import models

In [8]:
class r_net(nn.Module):
    def __init__(self):
        super(r_net, self).__init__()
        
        self.resnet = models.resnet34(pretrained=True)
        
        self.avgp = nn.AdaptiveAvgPool2d(1)
        self.final_conv = nn.Conv2d(512, 200, 1)
        
    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.avgp(x)
        x = self.final_conv(x)
        x = x.view(x.shape[0], -1)
        return x

In [9]:
r = r_net().cuda()

In [10]:
r(torch.Tensor(np.random.random((32, 3, 64, 64))).cuda()).shape

torch.Size([32, 200])

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(r.parameters())

In [ ]:
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        if i % 200 == 49: 
            print("-- ITERATION:", i)
        input, target = data

        # wrap input + target into variables
        input_var = input.cuda()
        target_var = target.cuda()
        
        #print(target_var)
            # compute output
        output = r(input_var)
        #print(output.shape)
        loss = criterion(output, target_var)
        
        # computer gradient + sgd step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print progress
        running_loss += loss.item()
            
            
        if i % 200 == 49:  # print every 2k mini-batches
            print(running_loss / 50)
            running_loss = 0.

-- ITERATION: 49
4.903546695709228
-- ITERATION: 249
15.153682508468627
